Natan Warelich 318159


# Wprowadzenie
Raport jest oparty na zbiorze win białych. Wykorzystany model do klasyfikacji to random forest, nastomiast do szacowania Extreme Gradient Boosting natomiast do podziału na grupy zastosowany będzie DBSCAN.

# Eksploracyjna analiza danych

## Weryfikacja danych

Zacznijmy od wczytania danych. Standardowo wykorzystano ku temu bibliotekę pandas.

In [49]:
import pandas as pd

data = pd.read_csv("white_wine.csv", sep=";")

Następnie wszystkie dane zostały sprawdzone czy mają prawidłowy typ danych. I okazało się że wszystkie zmienne są zdefiniowane jako object, przez zapis zmiennych przecinkowych przy zastosowania przecinka zamiast kropki. Dodatkowo niektóre dane mniejsze od zera zostały zapisane jako np ,4 co powodowało dodatkowe problemy. Przed edycją dane zostały sprawdzone pod względem wykrycia braków i żadne nie zostały wykryte. W celu zwalczenia tego zastosowana została poniższa funkcja

In [51]:
for col in data.columns[:-1]:
    data[col] = data[col].astype(str)
    data[col] = data[col].apply(lambda x: "0" + x if x.startswith(",") else x)

    data[col] = data[col].str.replace(',', '.', regex=False)
    data[col] = data[col].astype(float).round(3)


Sprawdźmy teraz dane pod względem:
- duplikatów
- obserwacji odstających
- brak wartości niezgodnych z założeniami

Zacznijmy od braku duplikatów


In [52]:
print(data.duplicated().sum())

645


Jak widzimy mamy, aż 645 duplikatów, jest to spora liczba warto będzie sprawdzić, przy eksploracji danych różnicę między modelami z i bez tych danych.

Zobaczmy teraz czy wszystkie dane są zgodne z założeniami. Zacznijmy od danych ujemnych.

In [53]:
print(data[data <= 0].sum())

fixedacidity          0.0
volatileacidity       0.0
citricacid            0.0
residualsugar         0.0
chlorides             0.0
freesulfurdioxide     0.0
totalsulfurdioxide    0.0
density               0.0
pH                    0.0
sulphates             0.0
alcohol               0.0
quality               0.0
dtype: float64


Dane ujemne nie występują. Sprawdźmy jeszcze czy gdzieś są dane są mało realistyczne między innymi:

- Alkohol: zwracamy uwagę na wartości powyżej 20%, które są nietypowe dla białego wina.
- pH: typowe wartości dla wina białego mieszczą się w przedziale [3.0, 3.4]. Weryfikujemy, czy nie występują próbki o pH ≤ 2.5 lub pH ≥ 3.9, co może świadczyć o błędzie lub nietypowej próbce.
- Lotna kwasowość: zazwyczaj mieści się w przedziale [0.1, 1.5] g/l. Sprawdzamy, czy nie ma wartości ≥ 1.6 g/l, które mogą być podejrzane.
- Kwas cytrynowy: występuje zwykle w niewielkich ilościach, dlatego zwracamy uwagę na wartości ≥ 1.1 g/l.
- Cukier: ze względu na dużą zmienność między różnymi stylami wina, nie definiujemy tu wartości nietypowych.
- Chlorki: w białych winach mogą występować do około 0.6 g/l; sprawdzamy, czy nie pojawiają się wartości ≥ 1.0 g/l.
- Wolny i całkowity dwutlenek siarki: odstające wartości mogą wpływać na zdrowotność i jakość wina, ale nie zmieniają jego klasyfikacji jako białego wina.
- Gęstość: podobnie jak dwutlenek siarki, nie ma wartości dyskwalifikujących próbkę jako białe wino.

In [54]:
print(data['alcohol'].where(data['alcohol'] >= 20).count())
print(data['pH'].where((data['pH'] >= 3.9) | (data['pH'] <= 2.5)).count())
print(data['volatileacidity'].where((data['volatileacidity'] >= 1.6)).count())
print(data['citricacid'].where((data['citricacid'] >= 1.1)).count())
print(data['sulphates'].where((data['sulphates'] >= 1.0)).count())


0
0
0
0
2


Mamy jedynie dwa dane z chlorkach które można określić jako potencjalnie odstające, jednakże nie należy wydawać przedwcześnie osądów.



## Rozkład jakości win
Rozkład jakości win prezentuje się następująco:

![Wykres rozkładu jakości wina](rozklad_jakosci_wina.png)



Na podstawie wykresu można zauważyć, że rozkład zmiennej quality przypomina rozkład normalny w zakresie ocen od 1 do 7. Najwięcej jest win o jakości 4, natomiast najmniej o jakości 1 i 7. Jednakże widać, iż są to znikome wartości w związku z czym warto będzie je połaczyć z kategoriami obok. Taki rozkład może być korzystny, jeśli chcielibyśmy zastosować np. regresję liniową, choć należy pamiętać, że quality to zmienna dyskretna i porządkowa, więc model ten powinien być używany ostrożnie.

## Macierz korelacji i wykresy rozrzutu
Macierz korelacji liniowej Pearsona została przedstawiona na rysunku poniżej

![macierz korelacji](macierz_korelacji.png)

Na podstawie macierzy korelacji zauważalny jest praktycznie zerowy związek zmiennych free sulfur dioxide oraz citric acid z jakością wina. Najsilniejszą dodatnią korelację z jakością obserwujemy dla zawartości alkoholu, co sugeruje, że wyższy poziom alkoholu może być czynnikiem wpływającym pozytywnie na ocenę wina.

Warto również zwrócić uwagę na wyraźne ujemne korelacje pomiędzy alkoholem a innymi cechami, szczególnie z gęstością – co jest zgodne z fizyczną zależnością: im więcej alkoholu, tym niższa gęstość cieczy. Z kolei zależność między zawartością cukru a gęstością jest dodatnia – co także jest intuicyjne, ponieważ cukier zwiększa masę roztworu.

Aby lepiej zrozumieć te relacje, warto przeanalizować wykresy rozrzutu dla zmiennych wykazujących najsilniejszą korelację dodatnią i ujemną z jakością wina oraz z alkoholem.

![alkohol a wino](alkohol_a_wino.png)

Choć zawartość alkoholu wykazuje ogólną dodatnią korelację z jakością wina, nie ma tu jednej "złotej reguły". Z wykresu rozrzutu wynika, że wśród win ocenionych najwyżej, większość ma zawartość alkoholu w przedziale 12–13%, co może wskazywać na preferencję dla takiego balansu.

![alkohol a gęstość](alkohol_a_gestosc.png)

Zależność pomiędzy alkoholem a gęstością potwierdza założenia z macierzy korelacji – im wyższy procent alkoholu, tym niższa gęstość cieczy. Na wykresie zauważalna jest jedna wartość odstająca o stosunkowo wysokiej gęstości mimo stosunkowo dużej zawartości alkoholu.

![cukier a gestosc](cukier_a_gestosc.png)

Analiza tej obserwacji potwierdza przypuszczenie: wartość odstająca charakteryzuje się znaczną zawartością cukru resztkowego. Zgodnie z zasadami fizyki i chemii, większa ilość cukru zwiększa gęstość roztworu, co tłumaczy odchylenie od ogólnego trendu.

# Model klasyfikacyjny
## Model i parametry
Szkolony model to Random Forest, skorzystamy z następujących ustawień parametrów:
- test_size = 0.3
- random_state = 318159
- n_estimators = 300
## Bazowy model
Statystyki bazowego modelu prezentują się następująco.

In [92]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, mean_absolute_error
import numpy as np

X = data.iloc[:, :-1]
y = data.iloc[:, -1]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=318159)

clf = RandomForestClassifier(n_estimators=300, random_state=318159)
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)


acc = accuracy_score(y_test, y_pred)


within_1 = np.abs(y_test - y_pred) <= 1
acc_within_1 = np.mean(within_1)


mae = mean_absolute_error(y_test, y_pred)

print("Accuracy:", acc)
print("Accuracy with tolerance ±1:", acc_within_1)
print("Mean Absolute Error (MAE):", mae)
print("\nClassification Report:\n", classification_report(y_test, y_pred, zero_division=0))

Accuracy: 0.6963151207115629
Accuracy with tolerance ±1: 0.9720457433290979
Mean Absolute Error (MAE): 0.3341804320203304

Classification Report:
               precision    recall  f1-score   support

           1       0.00      0.00      0.00         1
           2       0.89      0.29      0.43        28
           3       0.67      0.71      0.69       252
           4       0.68      0.75      0.72       357
           5       0.75      0.65      0.70       124
           6       1.00      0.46      0.63        24
           7       0.00      0.00      0.00         1

    accuracy                           0.70       787
   macro avg       0.57      0.41      0.45       787
weighted avg       0.71      0.70      0.69       787



Tak jak przypuszczaliśmy bazowy model nie wykrywa 1 i 7 w związku z czym połączmy te zmienne z kategoriami obok. Natomiast bazowy model ma trafność 65% co nie należyt do najlepszych wyników. Jednakże warto zwrócić uwagę na trafność z tolerancją ±1. Jest ona bardzo wysoka co się zgadza gdyż MAE wynosi ok 0.33. Sprawdźmy rezultaty dla połączonych danych, oraz pozbądźmy się tych danych których korelacja wskazuje na nikły wpływ na miarę jakości modelu, a dokładniej: citricacid, freesulfurdioxide, sulphates.

In [94]:

X_filtered = X.drop(columns=["citricacid", "freesulfurdioxide", "sulphates"])
y_grouped = y.replace({1: 2, 7: 6})

X_train, X_test, y_train, y_test = train_test_split(X_filtered, y_grouped, test_size=0.2, random_state=318159)

clf = RandomForestClassifier(n_estimators=300, random_state=318159)
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)


acc = accuracy_score(y_test, y_pred)

within_1 = np.abs(y_test - y_pred) <= 1
acc_within_1 = np.mean(within_1)

mae = mean_absolute_error(y_test, y_pred)

print("Accuracy:", acc)
print("Accuracy with tolerance ±1:", acc_within_1)
print("Mean Absolute Error (MAE):", mae)
print("\nClassification Report:\n", classification_report(y_test, y_pred, zero_division=0))

Accuracy: 0.6759847522236341
Accuracy with tolerance ±1: 0.9733163913595934
Mean Absolute Error (MAE): 0.3519695044472681

Classification Report:
               precision    recall  f1-score   support

           2       0.75      0.21      0.32        29
           3       0.68      0.69      0.69       252
           4       0.67      0.73      0.70       357
           5       0.65      0.65      0.65       124
           6       0.92      0.44      0.59        25

    accuracy                           0.68       787
   macro avg       0.73      0.54      0.59       787
weighted avg       0.68      0.68      0.67       787



Połączenie co raczej jest oczywiste tylko obniżyło nam notowania. Jednakże, nasza próbka danych jest tak niska, że i tak warto je zachować. Sprawdźmy więc teraz model wykluczając duplikaty a następnie outlinery.

In [121]:
data_cpy = data.copy()
data_cpy = data_cpy.drop_duplicates()

X = data_cpy.iloc[:, :-1]
y = data_cpy.iloc[:, -1]

y_grouped = y.replace({1: 2, 7: 6})

X_train, X_test, y_train, y_test = train_test_split(X, y_grouped, test_size=0.2, random_state=318159)

clf = RandomForestClassifier(n_estimators=300, random_state=318159)
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

acc = accuracy_score(y_test, y_pred)

within_1 = np.abs(y_test - y_pred) <= 1
acc_within_1 = np.mean(within_1)

mae = mean_absolute_error(y_test, y_pred)

print("Accuracy:", acc)
print("Accuracy with tolerance ±1:", acc_within_1)
print("Mean Absolute Error (MAE):", mae)
print("\nClassification Report:\n", classification_report(y_test, y_pred, zero_division=0))

Accuracy: 0.5486322188449848
Accuracy with tolerance ±1: 0.9513677811550152
Mean Absolute Error (MAE): 0.5015197568389058

Classification Report:
               precision    recall  f1-score   support

           2       0.80      0.11      0.19        37
           3       0.57      0.59      0.58       185
           4       0.54      0.72      0.62       288
           5       0.54      0.29      0.38       126
           6       0.50      0.09      0.15        22

    accuracy                           0.55       658
   macro avg       0.59      0.36      0.38       658
weighted avg       0.56      0.55      0.52       658



Pozbycie się duplikatów znacząco pogorszyło nasz model w związku z czym dalsze prace będą uwzględniałe te dane. Sprawdźmy teraz sytuację bez outlinerów.

In [95]:
data_cpy = data.copy()

X = data_cpy.iloc[:, :-1]
y = data_cpy.iloc[:, -1]

y_grouped = y.replace({1: 2, 7: 6})

Q1 = X.quantile(0.25)
Q3 = X.quantile(0.75)
IQR = Q3 - Q1

mask = ~((X < (Q1 - 1.5 * IQR)) | (X > (Q3 + 1.5 * IQR))).any(axis=1)

X_no_outliers = X[mask]
y_no_outliers = y_grouped[mask]

X_train, X_test, y_train, y_test = train_test_split(X_no_outliers, y_no_outliers, test_size=0.3, random_state=318159)

clf = RandomForestClassifier(n_estimators=300, random_state=318159)
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)


acc = accuracy_score(y_test, y_pred)

within_1 = np.abs(y_test - y_pred) <= 1
acc_within_1 = np.mean(within_1)

mae = mean_absolute_error(y_test, y_pred)

print("Accuracy:", acc)
print("Accuracy with tolerance ±1:", acc_within_1)
print("Mean Absolute Error (MAE):", mae)
print("\nClassification Report:\n", classification_report(y_test, y_pred, zero_division=0))

Accuracy: 0.6793422404933196
Accuracy with tolerance ±1: 0.9681397738951696
Mean Absolute Error (MAE): 0.35354573484069884

Classification Report:
               precision    recall  f1-score   support

           2       1.00      0.26      0.41        23
           3       0.72      0.66      0.68       276
           4       0.64      0.82      0.72       445
           5       0.73      0.50      0.60       195
           6       0.85      0.32      0.47        34

    accuracy                           0.68       973
   macro avg       0.79      0.51      0.58       973
weighted avg       0.70      0.68      0.67       973



Brak outlinerów wreszcie pozwolił nam uzyskać lepsze wyniki. Widzimy również lekkie przecuczenie dla 1-2. Jednakże sprawdzenie na innych parametrach powodowało obniżenie recall oraz f1-score. Co ciekawe trafność ogólnego modelu wzrosła, jednakże MAE oraz dla tolerancji spadła. Teraz spróbujmy polepszyć jakość tego modelu poprzez zastosowanie danych walidacyjnych.

In [96]:

data_cpy = data.copy()

X = data_cpy.iloc[:, :-1]
y = data_cpy.iloc[:, -1]

y_grouped = y.replace({1: 2, 7: 6})

Q1 = X.quantile(0.25)
Q3 = X.quantile(0.75)
IQR = Q3 - Q1

mask = ~((X < (Q1 - 1.5 * IQR)) | (X > (Q3 + 1.5 * IQR))).any(axis=1)

X_no_outliers = X[mask]
y_no_outliers = y_grouped[mask]

# Podział na trening (60%), walidację (20%) i test (20%) ze stratifikacją
X_train, X_temp, y_train, y_temp = train_test_split(
    X_no_outliers, y_no_outliers, test_size=0.4, random_state=318159, stratify=y_no_outliers
)

X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.5, random_state=318159, stratify=y_temp
)

clf = RandomForestClassifier(n_estimators=300, random_state=318159)
clf.fit(X_train, y_train)

# Predykcje na zbiorze walidacyjnym
y_val_pred = clf.predict(X_val)

val_acc = accuracy_score(y_val, y_val_pred)
val_acc_within_1 = np.mean(np.abs(y_val - y_val_pred) <= 1)
val_mae = mean_absolute_error(y_val, y_val_pred)

print("Validation Accuracy:", val_acc)
print("Validation Accuracy with tolerance ±1:", val_acc_within_1)
print("Validation MAE:", val_mae)
print("Validation Classification Report:\n", classification_report(y_val, y_val_pred, zero_division=0))


# Predykcje na zbiorze testowym
y_test_pred = clf.predict(X_test)

test_acc = accuracy_score(y_test, y_test_pred)
test_acc_within_1 = np.mean(np.abs(y_test - y_test_pred) <= 1)
test_mae = mean_absolute_error(y_test, y_test_pred)

print("\nTest Accuracy:", test_acc)
print("Test Accuracy with tolerance ±1:", test_acc_within_1)
print("Test MAE:", test_mae)
print("Test Classification Report:\n", classification_report(y_test, y_test_pred, zero_division=0))

Validation Accuracy: 0.6178736517719569
Validation Accuracy with tolerance ±1: 0.9661016949152542
Validation MAE: 0.41602465331278893
Validation Classification Report:
               precision    recall  f1-score   support

           2       1.00      0.24      0.38        17
           3       0.65      0.60      0.62       184
           4       0.60      0.72      0.66       298
           5       0.59      0.51      0.55       128
           6       1.00      0.27      0.43        22

    accuracy                           0.62       649
   macro avg       0.77      0.47      0.53       649
weighted avg       0.64      0.62      0.61       649


Test Accuracy: 0.6456086286594761
Test Accuracy with tolerance ±1: 0.9661016949152542
Test MAE: 0.3882896764252696
Test Classification Report:
               precision    recall  f1-score   support

           2       0.60      0.19      0.29        16
           3       0.69      0.66      0.68       185
           4       0.62      0.76 

## Krzywa roc
Dane walidacyjne pogorszyły nasz model. W związku z czym sprawdziliśmy różne sposoby i otrzymaliśmy najlepszy wynik dla samego wyzbycia się outlinerów. Sprawdźmy więc jak wygląda krzywa roc dla tego modelu.

![krzywa ROC jakości wina](roc_jakosc_wina.png)

Jak widać najlepiej model znajduje klase 2 a najgorszej klasę 4 która jest najliczniejsza. Co się oczywiście zgadza z naszymi danymi, gdyż uwzględniamy tam recall i precision.

## Wpływ danych na budowę modelu
W przypadku tego modelu wykres najbardziej wpływowych danych prezentuje się następująco:

![ważność cech random forest](waznosc_cech_rf.png)

Jak widać tak jak współczynniki korelacji podpowiadały, wartość alkohol była najbardziej wpłwowa przy budowie modelu. Co ciekawe jednak gęstość która miała być jedną z bardziej wpływowych wartości w tym modelu została uznana za mało ważną. Oraz wolny dwutlenek siarki który wykazywał zerową korelację, był jedną z ważniejszych cech. Co tłumaczy dlaczego przy pozbyciu się danych z zerową korelacją jakość modelu spadała.



## Podsumowanie
Model nieradzi sobie najlepiej z klasyfikacją. Jednakże, w przypadku dania marginesu błędu +/- 1 model ten sprawdza się wyśmienicie co może stanowić rozwiązanie na postawiony problem. Dodatkowo najważniejszymi danymi przy budowie modelu niekoniecznie były te które pierwotnie wskazała korelacja Pearsona.

# Model Regresyjny
## Model i parametry
Wykorzystany model to XGBoost z następującymi parametrami:
- test_size = 0.3
- random_state = 318159
- n_estimators = 300

## Bazowy model
Statystyki dla bazowego modelu prezentują się następująco:

In [116]:
from xgboost import XGBRegressor

X = data_cpy.iloc[:, :-1]
y = data_cpy.iloc[:, -1]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=318159)

model = XGBRegressor(random_state=318159, n_estimators=300, eval_metric='mae')
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

y_pred_rounded = np.clip(np.round(y_pred), 1, 7).astype(int)

accuracy_exact = accuracy_score(y_test, y_pred_rounded)
accuracy_within_1 = np.mean(np.abs(y_test - y_pred_rounded) <= 1)
mae = mean_absolute_error(y_test, y_pred_rounded)

print(f"Accuracy (dokładna): {accuracy_exact:.4f}")
print(f"Accuracy z dopuszczalnym odstępstwem ±1: {accuracy_within_1:.4f}")
print(f"Mean Absolute Error (MAE): {mae:.4f}")

Accuracy (dokładna): 0.6224
Accuracy z dopuszczalnym odstępstwem ±1: 0.9594
Mean Absolute Error (MAE): 0.4217


Jak widzimy mamy w bardzo podobną sytuację, stosunkowo niską trafność, jednakże bardzo wysoką z dopuszczalnym błędem 1. Jako, iż wcześniej jedynym rozwiązaniem dającym jakiekolwiek rezultaty było pozbycie się danych odstających zróbmy to też tutaj.


In [124]:
data_cpy = data.copy()

X = data_cpy.iloc[:, :-1]
y = data_cpy.iloc[:, -1]

y_grouped = y.replace({1: 2, 7: 6})

Q1 = X.quantile(0.25)
Q3 = X.quantile(0.75)
IQR = Q3 - Q1

mask = ~((X < (Q1 - 1.5 * IQR)) | (X > (Q3 + 1.5 * IQR))).any(axis=1)

X_no_outliers = X[mask]
y_no_outliers = y_grouped[mask]

X_train, X_test, y_train, y_test = train_test_split(X_no_outliers, y_no_outliers, test_size=0.3, random_state=318159)

model = XGBRegressor(random_state=318159, n_estimators=300, eval_metric='mae')
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

y_pred_rounded = np.clip(np.round(y_pred), 1, 7).astype(int)

accuracy_exact = accuracy_score(y_test, y_pred_rounded)
accuracy_within_1 = np.mean(np.abs(y_test - y_pred_rounded) <= 1)
mae = mean_absolute_error(y_test, y_pred_rounded)

print(f"Accuracy (dokładna): {accuracy_exact:.4f}")
print(f"Accuracy z dopuszczalnym odstępstwem ±1: {accuracy_within_1:.4f}")
print(f"Mean Absolute Error (MAE): {mae:.4f}")

Accuracy (dokładna): 0.6536
Accuracy z dopuszczalnym odstępstwem ±1: 0.9723
Mean Absolute Error (MAE): 0.3751


Jak widać podobnie jak w przypadku klasyfikacji, usunięcie outlinerów dało pozytywne rezultaty. Jednakże jako, iż ten model jest zupełnie inny sprawdzone zostały rezultaty analogicznych podpunktów co w przypadku klasyfikacji, o to rezultaty:
- usuniecie zbędnych według korelacji danych: pogorszenie modelu
- pozbycie się duplikatów: pogorszenie modelu
- dane walidacyjne: pogorszenie modelu

## Wpływ danych na budowę modelu
W przypadku tego modelu wykres najbardziej wpływowych danych prezentuje się następująco:


![ważność cech xgboost](waznosc_cech_xgb.png)

Jak widać alkohol który wskazał wykres korelacji okazał się być drugą najmniej wpływową wartością przy budowie modelu. Najważniejszą wartością okazała się stała kwasowość, a następnie cukier. Stanowi to zupełnie inne wnioski, a niżeli te nasunięte przy pracy bezpośrednio z danymi.

## Podsumowanie
Model XGBoost dla regresji podobnie jak model random forest dla klasyfikacji nie jest najlepszy w przewidywaniu oceny jakości wina, jednakże dla wersji z marginesem błędu daje sobie radę. Jednakże w tym przypadku róznica między danymi wpłwowymi w budowie tego modelu jest znacząca gdyż dwie najbardziej zkorelowane z oceną jakości dane, miały najmniejszy wpływ na budowę modelu.

# Podział win na grupy

# Model i parametry
W celu pogrupowania danych wykorzystany zostanie model K-średnich, z następującymi parametrami:
- k = 7
- random_state = 318159
- n_components = 2


## Model z standaryzacją
Wykres modelu k-średnich ze standaryzacją prezentuje się następująco:

![k-średnich standaryzowane dane](ksrednich_standaryzacja.png)



Jak widać w przypadku grupowania metodą K-średnich dla danych poddanych standaryzacji dostajemy zupełnie sprzeczne z rzeczywistością informacje. Na wykresie możemy dostrzeć dużą ilość jedynek oraz dużą ilość siódemek co jest przeciwieństwem od faktycznych danych. Należy jeszcze zobaczyć miarę jakości, aby potwierdzić nasze informacje.

Tak jak przypuszczaliśmy model ten wręcz tragicznie przypisuje dane co widać na podstawie wykresu czy też średniej wartości miary sylwetki.
Sprawdźmy jak wygląda wykres gdy dane poddamy normalizacji.

## Model z normalizacją

![k-średnich normalizacja](ksrednich_normalizacja.png)

Oczywiście model K-Średnich źle dobiera podział na klasy. Można byłoby jeszcze sprawdzić dla mniejszych próbek danych. Z tym że ogólna ilość danych nie jest jakaś wielka, więc jedyne do czego to doprowadzi to do pogorszenia wyników. Zobaczmy jeszcze miarę jakości. zobaczmy jeszcze jak wygląda miara jakości.


![wykres miary sylwetki klastrów](miary_sylwetki_klastrow.png)

Potwierdza to nasze wnioski, model k-średnich źle rozdziela dane. Zobaczmy jeszcze jak to wygląda z danymi testowymi.

![wykres k-średnich danych testowych](ksrednich_dane_testowe.png)

Dane testowe oczywiście też niewpełni zgadzają się z rzeczywistością ze względu na specyfikację uczenia nienadzorowanego. Jednakże w tym przypadku śmiało możemy określić, iż model testowy jest o wiele bliżej prawdy, widać znacznie mniejszą ilość jakości 7, lecz modelowi temu dalej daleko do perfekcji.

# Podsumowanie
Ze względu na specyfikację danych, bardzo ciężko było zbudować model który by dobrze klasyfikował dane, szacował ich wartości końcowe, natomiast zarówno model XGBoost w regresji i Random Forest w klasyfikacji, przy daniu sobie współczynnika błędu ±1 miał bardzo wysoką trafność. Sytuacja najgorzej wyglądała w przypadku podziału na grupy. Algorytm K-Średnich nie był w stanie poprawnie porozdzielać grupy, co skutkowało znacznym zawyżaniem jakości 7.